In [5]:
# GHOST - Euler Test (2D)

import sys  
import numpy as np
import modepy as mp
sys.path.insert(0, '../src')
from Mesh import Mesh2D
from Discretization import SpatialDiscretization
from Solver import Solver

# discretization degree
p = 2

# geometry mapping degree (isoparametric)
p_geo = p

# read in mesh in GMSH format
mesh = Mesh2D("test", "../mesh/square_mesh_L10_x_11y_11.msh")

# set up periodic boundary conditions
left = np.array([1.0,0.0,0.0]) 
right = np.array([1.0,0.0,10.0])
bottom = np.array([0.0,1.0,0.0])
top = np.array([0.0,1.0,10.0])
mesh.add_bc_on_hyperplanes([left,right,bottom,top],[1,2,3,4])
mesh.make_periodic((1,2),[1]) # left-right periodic (bcs parallel to axis 1)
mesh.make_periodic((3,4),[0]) # top-bottom periodic (axis 0)

#curvilinear transformation used in Del Rey Fernandez et al. (2017)
mesh.map_mesh(f_map=Mesh2D.grid_transformation(warp_factor=0.2, L=10.0), p_geo=p_geo)

# volume and facet quadrature degrees
tau = 2*p
mu = 2*p-1

# solver parameters
params = {"project_title": "euler_p2cpt3",
         "problem": "compressible_euler",
         "specific_heat_ratio": 1.4,
         "numerical_flux": "roe",
         "initial_condition": "isentropic_vortex",
         "initial_vortex_centre": np.array([5.0,5.0]),
         "background_temperature": 1.0,
         "background_velocity": np.array([1.0,1.0]),
         "integration_type": "quadrature",
         "solution_degree": p,
         "volume_quadrature_degree": tau,
         "facet_quadrature_degree": mu,
         "facet_rule": "lgl",
         "solution_representation": "modal",
         "form": "weak",
         "correction": "c_+",
         "time_integrator": "rk44",
         "final_time": 1.0,
         "time_step_scale": 0.005}

# set up solver
solver = Solver(params,mesh)
weak = Solver(params,mesh)
params_strong = params.copy()
params_strong["form"] = "strong"
strong = Solver(params_strong,mesh)

In [6]:
strong.run(write_interval=0.01)
strong.post_process(error_quadrature_degree=4*p)

dt =  0.0005
writing every  20  time steps, total  2000
writing time step  20 : t =  0.010000000000000005
writing time step  40 : t =  0.020000000000000014
writing time step  60 : t =  0.030000000000000023
writing time step  80 : t =  0.04000000000000003
writing time step  100 : t =  0.05000000000000004
writing time step  120 : t =  0.060000000000000046
writing time step  140 : t =  0.07000000000000005
writing time step  160 : t =  0.08000000000000006
writing time step  180 : t =  0.09000000000000007
writing time step  200 : t =  0.10000000000000007
writing time step  220 : t =  0.11000000000000008
writing time step  240 : t =  0.12000000000000009
writing time step  260 : t =  0.1300000000000001
writing time step  280 : t =  0.1400000000000001
writing time step  300 : t =  0.1500000000000001
writing time step  320 : t =  0.16000000000000011
writing time step  340 : t =  0.17000000000000012
writing time step  360 : t =  0.18000000000000013
writing time step  380 : t =  0.190000000000000

In [ ]:
weak.run(write_interval=0.1)
weak.post_process(error_quadrature_degree=4*p)

In [ ]:
print("{:.3e}".format(strong.calculate_difference(weak)), "&",
        "{:.3e}".format(strong.I_f - strong.I_0), "&",        
        "{:.3e}".format(weak.I_f- weak.I_0), "&",        
        "{:.3e}".format(strong.calculate_error()), "&",
        "{:.3e}".format(weak.calculate_error()))

In [8]:
strong.I_0

array([ 98.24325055,  98.24492854,  98.24519713, 344.76721842])